# Reimagined-Vis

# v_a11y_lint

In [13]:
import altair as alt
from vega_datasets import data
import v_a11ylint as val
import test_fns as tf

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Example One: Not very accesibile visualization

In [2]:
tf.bad_theme()
tf.set_theme(tf.bad_theme)

In [3]:
basic_bar = tf.make_basic_bar()
basic_bar

alt.Chart(...)

In [4]:
val.run_lint(basic_bar)

{'color': {'text to background': 'colors too similar',
  'title color to background': 'colors too similar'},
 'font': {'axisX': {'titleFontSize': 'font size too small: font should be at least 16px'},
  'axisY': {'titleFontSize': 'font size too small: font should be at least 16px'},
  'text': {'fontSize': 'font size too small: font should be at least 16px'},
  'legend': {'titleFontSize': 'font size too small: font should be at least 16px'}},
 'title': 'Chart needs title'}

## Example Two: A Better Visualization

In [5]:
tf.better_theme()
tf.set_theme(tf.better_theme)

In [6]:
better_chart = tf.make_basic_bar('A vs. B')
better_chart

alt.Chart(...)

In [7]:
val.run_lint(better_chart)

{'color': {'text to background': 'colors too similar'},
 'font': {'legend': {'titleFontSize': 'font size too small: font should be at least 16px'},
  'title length': 'Chart title lacks description'}}

## Example Three: An Accessible Visualization

In [8]:
tf.best_theme()
tf.set_theme(tf.best_theme)

In [9]:
best_chart = tf.make_basic_bar('Variable A by Variable B: Category D Highest')
best_chart

alt.Chart(...)

In [10]:
val.run_lint(best_chart)

'Visualization is Accessible!'

## Your turn

### Set a theme

In [11]:
# to default theme
# tf.reset_theme()

In [14]:
# credit to Lilian Huang for this theme
# https://lilianhj.github.io/dataviz-static-portfolio/

# build/edit for your own theme
def class_theme():
    main_palette = ["#385ed4","#55b748","#db2b27","#b589da","#b75f31","#1696d2","#fdbf11","#ff1ae4"]
    sequential_palette = ["#cfe8f3","#aaecff","#a2d4ec","#73bfe2","#46abdb","#1696d2","#12719e","#0a4c6a","#062635"]
    return {
    "config": {
            "title": {
                "font": "Futura",
                "fontSize": 18,
                "anchor": "middle",
                "color": "darkblue",
            },
         "axisX": {
                "grid": False,
                "tickSize": 6,
             "labelFontSize": 10,
             "titleFontSize": 12,
         },
        "axisY": {
                "labelFontSize": 10,
                "tickSize": 6,
            "titleFontSize": 12,
         },
        "background": "white",
        "text": {
               "color": "#686863",
               "fontSize": 10,
               "fontWeight": 400,
            "baseline": "top",
            "filled": True,
            "lineBreak": "\n",
           },
            "bar": {
                "fill": "#1696d2"
            },
            "line": {
               "strokeWidth": 3,
           },
        "range": {
                "category": main_palette,
                "diverging": "blueorange",
            "ramp": sequential_palette,
            "ordinal": sequential_palette,
            "heatmap": sequential_palette
            },
        "legend": {
                "titleFontSize": 12
            }
    },
    }

alt.themes.register("my_custom_theme", class_theme)
alt.themes.enable("my_custom_theme")

ThemeRegistry.enable('my_custom_theme')

### Now, make a chart!
Find examples here: https://altair-viz.github.io/gallery/index.html

In [15]:
source = data.movies.url
# Top 10 movies by IMBD rating
test_chart = alt.Chart(
    source,
).mark_bar().encode(
    x=alt.X('Title:N', sort='-y'),
    y=alt.Y('IMDB_Rating:Q'),
    color=alt.Color('IMDB_Rating:Q')

).transform_window(
    rank='rank(IMDB_Rating)',
    sort=[alt.SortField('IMDB_Rating', order='descending')]
).transform_filter(
    (alt.datum.rank < 10)
)

test_chart

alt.Chart(...)

In [16]:
val.run_lint(test_chart)

{'color': {'text to background': 'colors too similar'},
 'font': {'axisX': {'titleFontSize': 'font size too small: font should be at least 16px'},
  'axisY': {'titleFontSize': 'font size too small: font should be at least 16px'},
  'text': {'fontSize': 'font size too small: font should be at least 16px'},
  'legend': {'titleFontSize': 'font size too small: font should be at least 16px'}},
 'title': 'Chart needs title'}